<a href="https://colab.research.google.com/github/NicolasFerre/ColabFerre/blob/dev/Archivo_de_descuentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerías necesarias

In [1571]:
'''!pip install xlsxwriter'''

'!pip install xlsxwriter'

In [1572]:
import pandas as pd
import numpy as np
import openpyxl
from datetime import datetime, timedelta
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
import xlsxwriter

# Archivos necesarios

In [1573]:
archivo = "resultados_descuentos.xlsx"

In [1574]:
descuentos = pd.read_excel('Descuentos.xlsx', sheet_name=0, header = 9)
descuentos.head()

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Documento,Unnamed: 1,Fecha,Unnamed: 3,Unnamed: 4,Folio,Unnamed: 6,Cliente,Unnamed: 8,Unnamed: 9,...,Unnamed: 28,Descuento $\n,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Total
0,Factura CFDI,NaN,2025-07-16 08:19:56,NaN,NaN,23114.0,NaN,APP INSULATORS DE MEXICO,NaN,NaN,...,NaN,$ 0.00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,$ 398.00
1,NaN,NaN,NaT,NaN,NaN,NaN,PZA,NaN,1.0,NaN,...,NaN,NaN,NaT,NaN,NaN,$ 195.00,NaN,NaN,NaN,NaN
2,NaN,NaN,NaT,NaN,NaN,NaN,PZA,NaN,1.0,NaN,...,NaN,NaN,NaT,NaN,NaN,$ 62.00,NaN,NaN,NaN,NaN
3,NaN,NaN,NaT,NaN,NaN,NaN,PZA,NaN,1.0,NaN,...,NaN,NaN,NaT,NaN,NaN,$ 99.00,NaN,NaN,NaN,NaN
4,NaN,NaN,NaT,NaN,NaN,NaN,PZA,NaN,3.0,NaN,...,NaN,NaN,NaT,NaN,NaN,$ 42.00,NaN,NaN,NaN,NaN


In [1575]:
ventas = pd.read_excel('Ventas.xlsx', sheet_name=0, header = 5)
ventas.head()

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Documento,Unnamed: 1,Unnamed: 2,Fecha,Unnamed: 4,Folio,Unnamed: 6,Cliente,Unnamed: 8,Unnamed: 9,...,Total,Unnamed: 31,Efectivo,Cheque,Crédito,Transferencia,Vales,Tarjeta,Anticipo,SICAR Pagos
0,Factura CFDI,NaN,NaN,2025-07-16 08:19:56,NaN,23114.0,NaN,APP INSULATORS DE MEXICO,NaN,NaN,...,$ 398.00,NaN,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 398.00,$ 0.00,$ 0.00
1,PZA,1.0,NaN,NaT,[7501206625378] 14137 Juego de 5 desarmadores ...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PZA,1.0,NaN,NaT,[7501206678121] 22625 Pinza de punta y corte 7...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PZA,1.0,NaN,NaT,[7501206644812] 22606 Pinza para electricista ...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PZA,3.0,NaN,NaT,[7506240610045] 11635 Disco Tipo 41 de 4-1/2' ...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Extraemos la columna Total y todas las formas de pago del archivo de ventas

In [1576]:
pagos = ventas.iloc[:,32:40]
pagos['Folio'] = ventas['Folio']
pagos.dropna(inplace=True)
pagos.reset_index(drop=True, inplace=True)
pagos.head()

,Efectivo,Cheque,Crédito,Transferencia,Vales,Tarjeta,Anticipo,SICAR Pagos,Folio
0,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 398.00,$ 0.00,$ 0.00,23114.0
1,"$ 1,370.00",$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,64301.0
2,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 504.00,$ 0.00,$ 0.00,23115.0
3,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 139.00,$ 0.00,$ 0.00,64302.0
4,$ 150.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,64303.0


Convertimos las columnas de total de ambos documentos en numérico y las unimos

In [1577]:
descuentos['Total'] = descuentos['Total'].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df = descuentos.merge(pagos, on='Folio', how='left')

# Limpieza de datos

Limpiamos los nulos de las columnas

In [1578]:
df['Documento'].fillna(0, inplace=True)
df['Folio'].fillna(0, inplace=True)
df['Fecha'].fillna(0, inplace=True)
df['Folio'].fillna(0, inplace=True)
df['Cliente'].fillna(0, inplace=True)
df['Caja'].fillna(0, inplace=True)
df['Usuario'].fillna(0, inplace=True)
df['Est'].fillna(0, inplace=True)
df.head()

/tmp/ipython-input-1578-1441296211.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Documento'].fillna(0, inplace=True)
/tmp/ipython-input-1578-1441296211.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

,Documento,Unnamed: 1,Fecha,Unnamed: 3,Unnamed: 4,Folio,Unnamed: 6,Cliente,Unnamed: 8,Unnamed: 9,...,Unnamed: 36,Total,Efectivo,Cheque,Crédito,Transferencia,Vales,Tarjeta,Anticipo,SICAR Pagos
0,Factura CFDI,NaN,2025-07-16 08:19:56,NaN,NaN,23114.0,NaN,APP INSULATORS DE MEXICO,NaN,NaN,...,NaN,398.0,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 398.00,$ 0.00,$ 0.00
1,0,NaN,0,NaN,NaN,0.0,PZA,0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,NaN,0,NaN,NaN,0.0,PZA,0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,NaN,0,NaN,NaN,0.0,PZA,0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,NaN,0,NaN,NaN,0.0,PZA,0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


El código generó duplicados por alguna razón (y no en todas las filas), así que los eliminamos

In [1579]:
df.drop_duplicates(df.loc[df['Folio'] != 0], keep='last', inplace=True)
df.head()

,Documento,Unnamed: 1,Fecha,Unnamed: 3,Unnamed: 4,Folio,Unnamed: 6,Cliente,Unnamed: 8,Unnamed: 9,...,Unnamed: 36,Total,Efectivo,Cheque,Crédito,Transferencia,Vales,Tarjeta,Anticipo,SICAR Pagos
0,Factura CFDI,NaN,2025-07-16 08:19:56,NaN,NaN,23114.0,NaN,APP INSULATORS DE MEXICO,NaN,NaN,...,NaN,398.0,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 398.00,$ 0.00,$ 0.00
1,0,NaN,0,NaN,NaN,0.0,PZA,0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,NaN,0,NaN,NaN,0.0,PZA,0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,NaN,0,NaN,NaN,0.0,PZA,0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,NaN,0,NaN,NaN,0.0,PZA,0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Eliminamos las columnas que no tienen ningun valor

In [1580]:
lista = ['Unnamed: 1', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 23', 'Unnamed: 25', 'Unnamed: 28', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 14', 'Unnamed: 13', 'Unnamed: 18', 'Unnamed: 27', 'Unnamed: 34', 'Unnamed: 9']
df.drop(lista, axis=1, inplace=True)
df.head()

,Documento,Fecha,Folio,Unnamed: 6,Cliente,Unnamed: 8,Unnamed: 10,Unnamed: 11,Unnamed: 12,Caja,...,Unnamed: 33,Total,Efectivo,Cheque,Crédito,Transferencia,Vales,Tarjeta,Anticipo,SICAR Pagos
0,Factura CFDI,2025-07-16 08:19:56,23114.0,NaN,APP INSULATORS DE MEXICO,NaN,NaN,NaN,NaN,Caja 1,...,NaN,398.0,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 398.00,$ 0.00,$ 0.00
1,0,0,0.0,PZA,0,1.0,[7501206625378] 14137 Juego de 5 desarmadores ...,NaN,NaN,0,...,$ 195.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,0.0,PZA,0,1.0,[7501206678121] 22625 Pinza de punta y corte 7...,NaN,NaN,0,...,$ 62.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,0.0,PZA,0,1.0,[7501206644812] 22606 Pinza para electricista ...,NaN,NaN,0,...,$ 99.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,0.0,PZA,0,3.0,[7506240610045] 11635 Disco Tipo 41 de 4-1/2' ...,NaN,NaN,0,...,$ 42.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Renombramos columnas que sí tienen datos importantes y limpiamos los nulos. Además, añadimos la columna de EAN

In [1581]:
df.rename(columns={'Unnamed: 6': 'Unidad', 'Unnamed: 10':'Producto', 'Unnamed: 11':"Clave", 'Unnamed: 12':'Descripción','Unnamed: 8':'Cantidad', 'Unnamed: 26':'Precio Unitario', 'Unnamed: 33':'Precio'}, inplace=True)
df['Producto'].fillna(0, inplace=True)
df['Clave'].fillna(0, inplace=True)
df['Descripción'].fillna(0, inplace=True)
df['Unidad'].fillna(" ", inplace=True)

df.insert(8, 'EAN', 0)
df.head()

/tmp/ipython-input-1581-1801825223.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Producto'].fillna(0, inplace=True)
/tmp/ipython-input-1581-1801825223.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

,Documento,Fecha,Folio,Unidad,Cliente,Cantidad,Producto,Clave,EAN,Descripción,...,Precio,Total,Efectivo,Cheque,Crédito,Transferencia,Vales,Tarjeta,Anticipo,SICAR Pagos
0,Factura CFDI,2025-07-16 08:19:56,23114.0,,APP INSULATORS DE MEXICO,NaN,0,0.0,0,0,...,NaN,398.0,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 398.00,$ 0.00,$ 0.00
1,0,0,0.0,PZA,0,1.0,[7501206625378] 14137 Juego de 5 desarmadores ...,0.0,0,0,...,$ 195.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,0.0,PZA,0,1.0,[7501206678121] 22625 Pinza de punta y corte 7...,0.0,0,0,...,$ 62.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,0.0,PZA,0,1.0,[7501206644812] 22606 Pinza para electricista ...,0.0,0,0,...,$ 99.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,0.0,PZA,0,3.0,[7506240610045] 11635 Disco Tipo 41 de 4-1/2' ...,0.0,0,0,...,$ 42.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Completamos los datos en las columnas de "Documento", "Fecha", "Cliente", etc.

In [1582]:
documento = df['Documento'][0]
for i in df.index:
  if df['Documento'][i] != 0:
    documento = df['Documento'][i]
  else:
    df.loc[i, 'Documento'] = documento
df.head()

,Documento,Fecha,Folio,Unidad,Cliente,Cantidad,Producto,Clave,EAN,Descripción,...,Precio,Total,Efectivo,Cheque,Crédito,Transferencia,Vales,Tarjeta,Anticipo,SICAR Pagos
0,Factura CFDI,2025-07-16 08:19:56,23114.0,,APP INSULATORS DE MEXICO,NaN,0,0.0,0,0,...,NaN,398.0,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 398.00,$ 0.00,$ 0.00
1,Factura CFDI,0,0.0,PZA,0,1.0,[7501206625378] 14137 Juego de 5 desarmadores ...,0.0,0,0,...,$ 195.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Factura CFDI,0,0.0,PZA,0,1.0,[7501206678121] 22625 Pinza de punta y corte 7...,0.0,0,0,...,$ 62.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Factura CFDI,0,0.0,PZA,0,1.0,[7501206644812] 22606 Pinza para electricista ...,0.0,0,0,...,$ 99.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Factura CFDI,0,0.0,PZA,0,3.0,[7506240610045] 11635 Disco Tipo 41 de 4-1/2' ...,0.0,0,0,...,$ 42.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1583]:
fecha = df['Fecha'][0]
for i in df.index:
  if df['Fecha'][i] != 0:
    fecha = df['Fecha'][i]
  else:
    df.loc[i, 'Fecha'] = fecha

In [1584]:
cliente = df['Cliente'][0]
for i in df.index:
  if df['Cliente'][i] != 0:
    cliente = df['Cliente'][i]
  else:
    df.loc[i, 'Cliente'] = cliente

caja = df['Caja'][0]
for i in df.index:
  if df['Caja'][i] != 0:
    caja = df['Caja'][i]
  else:
    df.loc[i, 'Caja'] = caja

usuario = df['Usuario'][0]
for i in df.index:
  if df['Usuario'][i] != 0:
    usuario = df['Usuario'][i]
  else:
    df.loc[i, 'Usuario'] = usuario

est = df['Est'][0]
for i in df.index:
  if df['Est'][i] != 0:
    est = df['Est'][i]
  else:
    df.loc[i, 'Est'] = est

folio = df['Folio'][0]
for i in df.index:
  if df['Folio'][i] != 0:
    folio = df['Folio'][i]
  else:
    df.loc[i, 'Folio'] = folio

Extraemos el EAN, la clave y la descripción del producto

In [1585]:
for i in df.index:
  producto = df['Producto'][i]
  if producto != 0:
    a = producto.split(']')
    ean = a[0][1:]
    b = a[1]
    for j in range(len(b)):
      if b[j].isalpha():
        clave = b[1:j]
        descripcion = b[j:]
        break
    df.loc[i, 'EAN'] = ean
    df.loc[i, 'Clave'] = clave
    df.loc[i, 'Descripción'] = descripcion

df.drop('Producto', axis=1, inplace=True)

/tmp/ipython-input-1585-934193665.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '7501206625378' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'EAN'] = ean
/tmp/ipython-input-1585-934193665.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '14137 ' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'Clave'] = clave


Limpiamos los datos de las columnas con datos financieros

In [1586]:
df['Efectivo'] = df["Efectivo"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df['Cheque'] = df["Cheque"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df['Crédito'] = df["Crédito"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df['Transferencia'] = df["Transferencia"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df['Vales'] = df["Vales"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df['Tarjeta'] = df["Tarjeta"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df['Anticipo'] = df["Anticipo"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df['SICAR Pagos'] = df["SICAR Pagos"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)

Eliminamos las filas que no tengan un producto asociado

In [1587]:
# Create a boolean mask for rows where 'Unidad' is one of the desired values
unidades = [' ', 'PZA', 'Set', 'PR', 'XFB', 'XRO', 'XKI', 'KG', 'KGM']
mask = df['Unidad'].isin(unidades)

# Drop rows where the 'Unidad' is NOT in the list of valid units
df.drop(df[~mask].index, inplace=True)

In [1588]:
mask

,Unidad
0,True
1,True
2,True
3,True
4,True
...,...
1311,True
1313,True
1315,True
1316,True


~mask: This is the inverse of the mask. Since mask is True for rows with valid units, ~mask is True for rows with invalid units.



In [1589]:
df['Precio Unitario'] = df['Precio Unitario'].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df['Precio'] = df['Precio'].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df.rename(columns={'Descuento $\n':'Descuento $'}, inplace=True)
df['Descuento $'] = df['Descuento $'].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
descuentos = df['Desc %'].str.strip().str.replace('%', '').astype(float)

Completamos la columna de "Total"

In [1590]:
df['Total'].fillna(0, inplace=True)
df['Precio'].fillna(0, inplace=True)
df['Total'] = df['Total'] + df['Precio']

/tmp/ipython-input-1590-547530952.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Total'].fillna(0, inplace=True)
/tmp/ipython-input-1590-547530952.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using '

In [1591]:
df['Efectivo'].fillna(-1, inplace=True)
df['Cheque'].fillna(-1, inplace=True)
df['Crédito'].fillna(-1, inplace=True)
df['Transferencia'].fillna(-1, inplace=True)
df['Vales'].fillna(-1, inplace=True)
df['Tarjeta'].fillna(-1, inplace=True)
df['Anticipo'].fillna(-1, inplace=True)
df['SICAR Pagos'].fillna(-1, inplace=True)

/tmp/ipython-input-1591-1653481481.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Efectivo'].fillna(-1, inplace=True)
/tmp/ipython-input-1591-1653481481.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

In [1592]:
efectivo = df['Efectivo'][0]
for i in df.index:
  if df['Efectivo'][i] != -1:
    if df['Efectivo'][i] == 0:
      efectivo = np.nan
    else:
      efectivo = 1
      df.loc[i, 'Efectivo'] = efectivo
  else:
    df.loc[i, 'Efectivo'] = efectivo

df['Efectivo'] = df['Efectivo'] * df['Total']

In [1593]:
cheque = df['Cheque'][0]
for i in df.index:
  if df['Cheque'][i] != -1:
    if df['Cheque'][i] == 0:
      cheque = np.nan
    else:
      cheque = 1
      df.loc[i, 'Cheque'] = cheque
  else:
    df.loc[i, 'Cheque'] = cheque

df['Cheque'] = df['Cheque'] * df['Total']

credito = df['Crédito'][0]
for i in df.index:
  if df['Crédito'][i] != -1:
    if df['Crédito'][i] == 0:
      credito = np.nan
    else:
      credito = 1
      df.loc[i, 'Crédito'] = credito
  else:
    df.loc[i, 'Crédito'] = credito

df['Crédito'] = df['Crédito'] * df['Total']

transferencia = df['Transferencia'][0]
for i in df.index:
  if df['Transferencia'][i] != -1:
    if df['Transferencia'][i] == 0:
      transferencia = np.nan
    else:
      transferencia = 1
      df.loc[i, 'Transferencia'] = transferencia
  else:
    df.loc[i, 'Transferencia'] = transferencia

df['Transferencia'] = df['Transferencia'] * df['Total']

vales = df['Vales'][0]
for i in df.index:
  if df['Vales'][i] != -1:
    if df['Vales'][i] == 0:
      vales = np.nan
    else:
      vales = 1
      df.loc[i, 'Vales'] = vales
  else:
    df.loc[i, 'Vales'] = vales

df['Vales'] = df['Vales'] * df['Total']

tarjeta = df['Tarjeta'][0]
for i in df.index:
  if df['Tarjeta'][i] != -1:
    if df['Tarjeta'][i] == 0:
      tarjeta = np.nan
    else:
      tarjeta = 1
      df.loc[i, 'Tarjeta'] = tarjeta
  else:
    df.loc[i, 'Tarjeta'] = tarjeta

df['Tarjeta'] = df['Tarjeta'] * df['Total']

anticipo = df['Anticipo'][0]
for i in df.index:
  if df['Anticipo'][i] != -1:
    if df['Anticipo'][i] == 0:
      anticipo = np.nan
    else:
      anticipo = 1
      df.loc[i, 'Anticipo'] = anticipo
  else:
    df.loc[i, 'Anticipo'] = anticipo

df['Anticipo'] = df['Anticipo'] * df['Total']

sicar_pagos = df['SICAR Pagos'][0]
for i in df.index:
  if df['SICAR Pagos'][i] != -1:
    if df['SICAR Pagos'][i] == 0:
      sicar_pagos = np.nan
    else:
      sicar_pagos = 1
      df.loc[i, 'SICAR Pagos'] = sicar_pagos
  else:
    df.loc[i, 'SICAR Pagos'] = sicar_pagos

df['SICAR Pagos'] = df['SICAR Pagos'] * df['Total']

In [1594]:
df['Desc %'].fillna(0, inplace=True)
descuento = df['Desc %'][0]
for i in df.index:
  if df['Desc %'][i] != 0:
    descuento = df['Desc %'][i]
  else:
    df.loc[i, 'Desc %'] = descuento

/tmp/ipython-input-1594-3336836358.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Desc %'].fillna(0, inplace=True)


Le damos mejor formato a las columnas

In [1595]:
df['Unidad'].replace(' ', '----------', inplace=True)
df['Cantidad'].fillna('----------', inplace=True)
df['Precio Unitario'].fillna('----------', inplace=True)
df['Precio'].replace(0, '----------', inplace=True)
df['Clave'].replace(0, '----------', inplace=True)
df['EAN'].replace(0, '----------', inplace=True)
df['Descripción'].replace(0, '----------', inplace=True)

/tmp/ipython-input-1595-566902907.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Unidad'].replace(' ', '----------', inplace=True)
/tmp/ipython-input-1595-566902907.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=T

Eliminamos las filas sin datos relevantes

In [1596]:
df.drop(labels=df.iloc[436:447,:].index, axis=0, inplace=True)

Separamos el dato de "Fecha" en Fecha y Hora

In [1597]:
df.insert(1, "Día", "")
df.insert(2, "Hora", "")

In [1598]:
df['Fecha'] = df['Fecha'].astype(str)

In [1599]:
for i in df.index:
  fecha = df['Fecha'][i].split(' ')
  df.loc[i, 'Día'] = fecha[0]
  df.loc[i, 'Hora'] = fecha[1]
df.drop('Fecha', axis=1, inplace=True)
df.rename(columns={'Día':'Fecha'}, inplace=True)

In [1600]:
descuentos = df[['Folio', 'Cliente','Clave', 'EAN', 'Desc %', 'Precio Unitario', 'Cantidad', 'Precio']]
descuentos.head(15)

,Folio,Cliente,Clave,EAN,Desc %,Precio Unitario,Cantidad,Precio
0,23114.0,APP INSULATORS DE MEXICO,----------,----------,0.00%,----------,----------,----------
1,23114.0,APP INSULATORS DE MEXICO,14137,7501206625378,0.00%,195.0,1.0,195.0
2,23114.0,APP INSULATORS DE MEXICO,22625,7501206678121,0.00%,62.0,1.0,62.0
3,23114.0,APP INSULATORS DE MEXICO,22606,7501206644812,0.00%,99.0,1.0,99.0
4,23114.0,APP INSULATORS DE MEXICO,11635,7506240610045,0.00%,14.0,3.0,42.0
10,64301.0,PUBLICO VIP,----------,----------,0.00%,----------,----------,----------
11,64301.0,PUBLICO VIP,101574,7506240686248,0.00%,75.0,1.0,75.0
12,64301.0,PUBLICO VIP,14991,7506240635161,0.00%,22.0,2.0,44.0
13,64301.0,PUBLICO VIP,13468,7506240653271,0.00%,29.0,3.0,87.0
14,64301.0,PUBLICO VIP,101977,7506240691747,0.00%,65.0,1.0,65.0


In [1601]:
descuentos = descuentos.loc[descuentos['Clave'] != '----------']
descuentos['Clave'].replace('', 0, inplace=True)
descuentos['EAN'] = descuentos['EAN'].astype(float)
descuentos.reset_index(drop=True, inplace=True)
descuentos.head()

/tmp/ipython-input-1601-2268923386.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  descuentos['Clave'].replace('', 0, inplace=True)
/tmp/ipython-input-1601-2268923386.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  descuentos['Clave'].replace('', 0, inplace=True)
/tmp/ipython-input-1601-2268923386.py:3: SettingWithCopyWarning: 
A value

,Folio,Cliente,Clave,EAN,Desc %,Precio Unitario,Cantidad,Precio
0,23114.0,APP INSULATORS DE MEXICO,14137,7.501207e+12,0.00%,195.0,1.0,195.0
1,23114.0,APP INSULATORS DE MEXICO,22625,7.501207e+12,0.00%,62.0,1.0,62.0
2,23114.0,APP INSULATORS DE MEXICO,22606,7.501207e+12,0.00%,99.0,1.0,99.0
3,23114.0,APP INSULATORS DE MEXICO,11635,7.506241e+12,0.00%,14.0,3.0,42.0
4,64301.0,PUBLICO VIP,101574,7.506241e+12,0.00%,75.0,1.0,75.0


In [1602]:
renglonaje = pd.read_excel("Renglonaje.xlsx", sheet_name=0)
renglonaje.head()

,código,clave,descripción,margen de mercado,caja,master,precio,unidad,ean,Total que paga Edy a Truper contando reserva en garantía CON IVA,...,precio público sin IVA,Marca,Precio Medio Mayoreo sin IVA,Precio Medio Mayoreo con IVA,Codigo SAT,Descripcion SAT,Familia,Descripción Familia,Peso[Kg],Volumen[cm3]
0,100048,PET-15X,Llave ajustable (perico) 15' cromada mango de ...,MM00,4,12,585.0,Pieza,7.506241e+12,290.9246,...,504.31,Truper,461.21,535,27111707,Llaves ajustables,P085,Llaves ajustables,1.317,1660.968
1,100049,PET-15C,Llave ajustable (perico) 15' profesional croma...,MM00,4,12,825.0,Pieza,7.506241e+12,407.29444,...,711.21,Expert,642.24,745,27111707,Llaves ajustables,P085,Llaves ajustables,1.330,1660.968
2,100098,REP-CUT-5X,Estuche con 10 cuchillas SK2 de 9 mm para cutt...,MM00,6,144,28.0,Set,7.506241e+12,13.96797,...,24.14,Expert,22.41,26,27111503,Cuchillos de diversas aplicaciones,P049,Cutters y exactos,0.035,120.900
3,100099,REP-CUT-6X,Estuche con 10 cuchillas SK2 de 18 mm para cut...,MM00,6,144,49.0,Set,7.506241e+12,24.8253,...,42.24,Expert,39.66,46,27111503,Cuchillos de diversas aplicaciones,P049,Cutters y exactos,0.085,186.404
4,100101,REP-CUT-650,Estuche con 50 cuchillas SK4 de 18 mm para cut...,MM00,2,20,135.0,Set,7.506241e+12,65.94158,...,116.38,Truper,103.45,120,27111503,Cuchillos de diversas aplicaciones,P049,Cutters y exactos,0.368,425.700


In [1603]:
costos = renglonaje[['ean', 'precio mayoreo con IVA', 'precio mínimo de venta']]
costos.rename(columns={'ean':'EAN'}, inplace=True)
costos.head()

/tmp/ipython-input-1603-4132450909.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  costos.rename(columns={'ean':'EAN'}, inplace=True)


,EAN,precio mayoreo con IVA,precio mínimo de venta
0,7.506241e+12,485,375.0
1,7.506241e+12,685,525.0
2,7.506241e+12,23,18.0
3,7.506241e+12,42,32.0
4,7.506241e+12,110,85.0


In [1604]:
costos['precio mayoreo con IVA'].replace('*', 0)
costos['EAN'].fillna(0, inplace=True)
costos['EAN'].replace('', 0, inplace=True)
costos['EAN'].astype(int)
costos.head()

/tmp/ipython-input-1604-1881588175.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  costos['precio mayoreo con IVA'].replace('*', 0)
/tmp/ipython-input-1604-1881588175.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  costos['EAN'].fillna(0, inplace=True)
/tmp/ipython-input-1604-1881588175.py:2

,EAN,precio mayoreo con IVA,precio mínimo de venta
0,7.506241e+12,485,375.0
1,7.506241e+12,685,525.0
2,7.506241e+12,23,18.0
3,7.506241e+12,42,32.0
4,7.506241e+12,110,85.0


In [1605]:
descuentos = descuentos.merge(costos, on='EAN', how='left')
descuentos.head()

,Folio,Cliente,Clave,EAN,Desc %,Precio Unitario,Cantidad,Precio,precio mayoreo con IVA,precio mínimo de venta
0,23114.0,APP INSULATORS DE MEXICO,14137,7.501207e+12,0.00%,195.0,1.0,195.0,235,195.0
1,23114.0,APP INSULATORS DE MEXICO,22625,7.501207e+12,0.00%,62.0,1.0,62.0,75,62.0
2,23114.0,APP INSULATORS DE MEXICO,22606,7.501207e+12,0.00%,99.0,1.0,99.0,119,99.0
3,23114.0,APP INSULATORS DE MEXICO,11635,7.506241e+12,0.00%,14.0,3.0,42.0,18,14.0
4,64301.0,PUBLICO VIP,101574,7.506241e+12,0.00%,75.0,1.0,75.0,89,75.0


In [1606]:
descuentos['Diferencia mayoreo'] = descuentos['Precio Unitario'] - descuentos['precio mayoreo con IVA']
descuentos['Diferencia minimo'] = descuentos['Precio Unitario'] - descuentos['precio mínimo de venta']
descuentos.head()

,Folio,Cliente,Clave,EAN,Desc %,Precio Unitario,Cantidad,Precio,precio mayoreo con IVA,precio mínimo de venta,Diferencia mayoreo,Diferencia minimo
0,23114.0,APP INSULATORS DE MEXICO,14137,7.501207e+12,0.00%,195.0,1.0,195.0,235,195.0,-40.0,0.0
1,23114.0,APP INSULATORS DE MEXICO,22625,7.501207e+12,0.00%,62.0,1.0,62.0,75,62.0,-13.0,0.0
2,23114.0,APP INSULATORS DE MEXICO,22606,7.501207e+12,0.00%,99.0,1.0,99.0,119,99.0,-20.0,0.0
3,23114.0,APP INSULATORS DE MEXICO,11635,7.506241e+12,0.00%,14.0,3.0,42.0,18,14.0,-4.0,0.0
4,64301.0,PUBLICO VIP,101574,7.506241e+12,0.00%,75.0,1.0,75.0,89,75.0,-14.0,0.0


Creación del excel

In [1607]:
writer = pd.ExcelWriter(archivo, engine="xlsxwriter")

In [1608]:
df.to_excel(writer, sheet_name='SICAR', index=False)
descuentos.to_excel(writer, sheet_name='Reporte', index=False)

In [1609]:
writer.close()

# Dando formato al Excel

In [1610]:
wb = load_workbook(archivo)
ws = wb.get_sheet_by_name('SICAR')

/tmp/ipython-input-1610-3920817291.py:2: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  ws = wb.get_sheet_by_name('SICAR')


In [1611]:
from ctypes import alignment
font_t = Font(name='Comfortaa', size=12, bold=True, italic=False, color='ffffff')
font = Font(name='Comfortaa', size=11, bold=False, italic=False, color='000000')

In [1612]:
fill_b = PatternFill(fill_type="solid", start_color="000000", end_color="000000")
fill_r = PatternFill(fill_type="solid", start_color="ffc7ce", end_color="ffc7ce")
fill_g = PatternFill(fill_type="solid", start_color="b6d7a8", end_color="b6d7a8")
fill_y = PatternFill(fill_type="solid", start_color="ffe599", end_color="ffe599")
fill_p = PatternFill(fill_type="solid", start_color="8e7cc3", end_color="8e7cc3")

In [1613]:
for col in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']:
    ws.column_dimensions[col].width = 18

ws.column_dimensions['F'].width = 40

In [1614]:
for col in range(1, 27):
  ws.cell(1, col).font = font_t
  ws.cell(1, col).alignment = Alignment(horizontal='center')
  ws.cell(1, col).fill = fill_b

for row in range(2, 1000):
  for col in range(1, 27):
    ws.cell(row=row, column=col).font = font
    ws.cell(row=row, column=col).alignment = Alignment(horizontal='center')

for row in range(2, 1000):
  ws.cell(row=row, column=18).fill = fill_g

In [1615]:
num_form = '$ #,##0.00'
for row in range(2, 1000):
  for col in range(15, 28):
    ws.cell(row=row, column=col).number_format = num_form

In [1616]:
wb.save(archivo)

In [1617]:
ws = wb.get_sheet_by_name('Reporte')


/tmp/ipython-input-1617-1864462614.py:1: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  ws = wb.get_sheet_by_name('Reporte')


In [1618]:
for col in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M']:
    ws.column_dimensions[col].width = 18

In [1619]:
for col in range(1, 13):
  ws.cell(1, col).font = font_t
  ws.cell(1, col).alignment = Alignment(horizontal='right')
  ws.cell(1, col).fill = fill_b

for row in range(2, 1000):
  for col in range(1, 13):
    ws.cell(row=row, column=col).font = font
    ws.cell(row=row, column=col).alignment = Alignment(horizontal='center')

In [1620]:
for row in range(2, 1000):
  if(ws.cell(row=row, column=1).value == None):
    break
  if(ws.cell(row=row, column=11).value != 0) and (ws.cell(row=row, column=12).value != 0):
    if(ws.cell(row=row, column=5).value == '0.00%'):
      if(ws.cell(row=row, column=3).value != 0):
        for col in range(1, 13):
          ws.cell(row=row, column=col).fill = fill_r
      else:
        for col in range(1, 13):
          ws.cell(row=row, column=col).fill = fill_p
    else:
      for col in range(1, 13):
        ws.cell(row=row, column=col).fill = fill_y

In [1621]:
num_form = '$ #,##0.00'
for row in range(2, 1000):
  ws.cell(row=row, column=6).number_format = num_form
  for col in range(8, 13):
    ws.cell(row=row, column=col).number_format = num_form


In [1622]:
wb.save(archivo)